In [1]:
import os
from pathlib import Path
from dotenv import load_dotenv

# Load .env file from the same directory as this notebook
# In Jupyter, the current working directory is typically where the notebook is located
env_file = Path('.env')
if env_file.exists():
    load_dotenv(dotenv_path=env_file, override=True)
    print(f"✅ Loading .env file from: {env_file.absolute()}")
else:
    # If not found, try loading from current directory (default behavior)
    load_dotenv(override=True)
    print(f"⚠️  .env file not found at {env_file.absolute()}, trying default location...")

try:
    # Check if GEMINI_API_KEY exists in system environment
    gemini_key = os.getenv("GEMINI_API_KEY")
    if gemini_key:
        print(f"ℹ️  Note: GEMINI_API_KEY found in system environment (will be unset to avoid conflicts)")
        # Unset GEMINI_API_KEY to avoid any confusion - ADK should use GOOGLE_API_KEY
        if "GEMINI_API_KEY" in os.environ:
            del os.environ["GEMINI_API_KEY"]
    
    # Load GOOGLE_API_KEY from .env file
    GOOGLE_API_KEY = os.getenv("GOOGLE_API_KEY")
    if not GOOGLE_API_KEY:
        raise ValueError("GOOGLE_API_KEY not found in .env file")
    
    # Explicitly set environment variables for ADK (this ensures it's available)
    os.environ["GOOGLE_API_KEY"] = GOOGLE_API_KEY
    os.environ["GOOGLE_GENAI_USE_VERTEXAI"] = "FALSE"
    
    # Verify API key format (basic check - starts with AIza)
    if not GOOGLE_API_KEY.startswith("AIza"):
        print("⚠️  Warning: API key format looks unusual. Make sure it's a valid Gemini API key.")
        raise ValueError("API key format is invalid - should start with 'AIza'")
    
    # Verify the key is actually set in environment
    env_key = os.getenv("GOOGLE_API_KEY")
    if env_key != GOOGLE_API_KEY:
        raise ValueError(f"API key mismatch! Expected {GOOGLE_API_KEY[:10]}... but got {env_key[:10] if env_key else 'None'}...")
    
    print(f"✅ Gemini API key loaded from .env file (first 10 chars: {GOOGLE_API_KEY[:10]}...)")
    print(f"✅ Verified: GOOGLE_API_KEY is set in environment (length: {len(GOOGLE_API_KEY)} chars)")
    print("ℹ️  ADK will use GOOGLE_API_KEY")
except Exception as e:
    print(f"🔑 Authentication Error: Please make sure you have added 'GOOGLE_API_KEY' to your .env file. Details: {e}")
    raise

✅ Loading .env file from: /Users/ayoushchourasia/Documents/learning/5-Day-AI-Agents-Intensive-Course-with-Google/Day 1/notebooks/local_exercise/.env
ℹ️  Note: GEMINI_API_KEY found in system environment (will be unset to avoid conflicts)
✅ Gemini API key loaded from .env file (first 10 chars: AIzaSyCxBj...)
✅ Verified: GOOGLE_API_KEY is set in environment (length: 39 chars)
ℹ️  ADK will use GOOGLE_API_KEY


In [2]:
from google.adk.agents import Agent, SequentialAgent, ParallelAgent, LoopAgent
from google.adk.runners import InMemoryRunner
from google.adk.tools import AgentTool, FunctionTool, google_search
from google.genai import types

print("✅ ADK components imported successfully.")

✅ ADK components imported successfully.


In [3]:
# Research Agent: Its job is to use the google_search tool and present findings.
research_agent = Agent(
    name="ResearchAgent",
    model="gemini-2.5-flash-lite",
    instruction="""You are a specialized research agent. Your only job is to use the
    google_search tool to find 2-3 pieces of relevant information on the given topic and present the findings with citations.""",
    tools=[google_search],
    output_key="research_findings", # The result of this agent will be stored in the session state with this key.
)

print("✅ research_agent created.")

✅ research_agent created.


In [4]:
# Summarizer Agent: Its job is to summarize the text it receives.
summarizer_agent = Agent(
    name="SummarizerAgent",
    model="gemini-2.5-flash-lite",
    # The instruction is modified to request a bulleted list for a clear output format.
    instruction="""Read the provided research findings: {research_findings}
Create a concise summary as a bulleted list with 3-5 key points.""",
    output_key="final_summary",
)

print("✅ summarizer_agent created.")

✅ summarizer_agent created.


In [5]:
# Root Coordinator: Orchestrates the workflow by calling the sub-agents as tools.
root_agent = Agent(
    name="ResearchCoordinator",
    model="gemini-2.5-flash-lite",
    # This instruction tells the root agent HOW to use its tools (which are the other agents).
    instruction="""You are a research coordinator. Your goal is to answer the user's query by orchestrating a workflow.
1. First, you MUST call the `ResearchAgent` tool to find relevant information on the topic provided by the user.
2. Next, after receiving the research findings, you MUST call the `SummarizerAgent` tool to create a concise summary.
3. Finally, present the final summary clearly to the user as your response.""",
    # We wrap the sub-agents in `AgentTool` to make them callable tools for the root agent.
    tools=[
        AgentTool(research_agent),
        AgentTool(summarizer_agent)
    ],
)

print("✅ root_agent created.")

✅ root_agent created.


In [6]:
runner = InMemoryRunner(agent=root_agent)
response = await runner.run_debug("What are the latest advancements in quantum computing and what do they mean for AI?")


 ### Created new session: debug_session_id

User > What are the latest advancements in quantum computing and what do they mean for AI?


ResearchCoordinator > Quantum computers, with their exponential processing power from qubits, are poised to overcome classical AI limitations, leading to a field known as "Quantum AI" that promises unprecedented performance.

Key AI enhancements include accelerated training and optimization, the ability to tackle problems intractable for classical computers, improved Natural Language Processing, more efficient data processing, and the development of more powerful AI models. Quantum AI also has the potential for significant energy efficiency gains, and hybrid quantum-classical solutions are expected to be crucial for near-term practical applications.

Major companies are investing in Quantum AI, and while widespread adoption is years away, the market is projected to reach tens of billions of dollars by 2035, with Quantum AI being a primary growth driver.


In [7]:
# Outline Agent: Creates the initial blog post outline.
outline_agent = Agent(
    name="OutlineAgent",
    model="gemini-2.5-flash-lite",
    instruction="""Create a blog outline for the given topic with:
    1. A catchy headline
    2. An introduction hook
    3. 3-5 main sections with 2-3 bullet points for each
    4. A concluding thought""",
    output_key="blog_outline", # The result of this agent will be stored in the session state with this key.
)

print("✅ outline_agent created.")

✅ outline_agent created.


In [8]:
# Writer Agent: Writes the full blog post based on the outline from the previous agent.
writer_agent = Agent(
    name="WriterAgent",
    model="gemini-2.5-flash-lite",
    # The `{blog_outline}` placeholder automatically injects the state value from the previous agent's output.
    instruction="""Following this outline strictly: {blog_outline}
    Write a brief, 200 to 300-word blog post with an engaging and informative tone.""",
    output_key="blog_draft", # The result of this agent will be stored with this key.
)

print("✅ writer_agent created.")

✅ writer_agent created.


In [9]:
# Editor Agent: Edits and polishes the draft from the writer agent.
editor_agent = Agent(
    name="EditorAgent",
    model="gemini-2.5-flash-lite",
    # This agent receives the `{blog_draft}` from the writer agent's output.
    instruction="""Edit this draft: {blog_draft}
    Your task is to polish the text by fixing any grammatical errors, improving the flow and sentence structure, and enhancing overall clarity.""",
    output_key="final_blog", # This is the final output of the entire pipeline.
)

print("✅ editor_agent created.")

✅ editor_agent created.


In [10]:
root_agent = SequentialAgent(
    name="BlogPipeline",
    sub_agents=[outline_agent, writer_agent, editor_agent],
)

print("✅ Sequential Agent created.")

✅ Sequential Agent created.


In [11]:
runner = InMemoryRunner(agent=root_agent)
response = await runner.run_debug("Write a blog post about the benefits of multi-agent systems for software developers")


 ### Created new session: debug_session_id

User > Write a blog post about the benefits of multi-agent systems for software developers
OutlineAgent > ## Outline:

**Headline:** Supercharge Your Code: How Multi-Agent Systems Are Revolutionizing Software Development

**Introduction Hook:**
*   Imagine a team of highly specialized, intelligent assistants working tirelessly behind the scenes of your development workflow. This isn't science fiction anymore. Multi-agent systems (MAS) are emerging as a powerful paradigm, offering significant advantages for software developers looking to boost efficiency, innovation, and code quality.

**Main Sections:**

**1. Enhanced Problem Solving & Complexity Management**
    *   **Decomposition into manageable parts:** MAS allows breaking down complex software problems into smaller, independent agents, each responsible for a specific task or sub-problem. This makes development more modular and easier to reason about.
    *   **Specialization and experti

In [12]:
# Tech Researcher: Focuses on AI and ML trends.
tech_researcher = Agent(
    name="TechResearcher",
    model="gemini-2.5-flash-lite",
    instruction="""Research the latest AI/ML trends. Include 3 key developments,
the main companies involved, and the potential impact. Keep the report very concise (100 words).""",
    tools=[google_search],
    output_key="tech_research", # The result of this agent will be stored in the session state with this key.
)

print("✅ tech_researcher created.")

✅ tech_researcher created.


In [13]:
# Health Researcher: Focuses on medical breakthroughs.
health_researcher = Agent(
    name="HealthResearcher",
    model="gemini-2.5-flash-lite",
    instruction="""Research recent medical breakthroughs. Include 3 significant advances,
their practical applications, and estimated timelines. Keep the report concise (100 words).""",
    tools=[google_search],
    output_key="health_research", # The result will be stored with this key.
)

print("✅ health_researcher created.")

✅ health_researcher created.


In [14]:
# Finance Researcher: Focuses on fintech trends.
finance_researcher = Agent(
    name="FinanceResearcher",
    model="gemini-2.5-flash-lite",
    instruction="""Research current fintech trends. Include 3 key trends,
their market implications, and the future outlook. Keep the report concise (100 words).""",
    tools=[google_search],
    output_key="finance_research", # The result will be stored with this key.
)

print("✅ finance_researcher created.")

✅ finance_researcher created.


In [15]:
# The AggregatorAgent runs *after* the parallel step to synthesize the results.
aggregator_agent = Agent(
    name="AggregatorAgent",
    model="gemini-2.5-flash-lite",
    # It uses placeholders to inject the outputs from the parallel agents, which are now in the session state.
    instruction="""Combine these three research findings into a single executive summary:

    **Technology Trends:**
    {tech_research}
    
    **Health Breakthroughs:**
    {health_research}
    
    **Finance Innovations:**
    {finance_research}
    
    Your summary should highlight common themes, surprising connections, and the most important key takeaways from all three reports. The final summary should be around 200 words.""",
    output_key="executive_summary", # This will be the final output of the entire system.
)

print("✅ aggregator_agent created.")

✅ aggregator_agent created.


In [16]:
# The ParallelAgent runs all its sub-agents simultaneously.
parallel_research_team = ParallelAgent(
    name="ParallelResearchTeam",
    sub_agents=[tech_researcher, health_researcher, finance_researcher],
)

# This SequentialAgent defines the high-level workflow: run the parallel team first, then run the aggregator.
root_agent = SequentialAgent(
    name="ResearchSystem",
    sub_agents=[parallel_research_team, aggregator_agent],
)

print("✅ Parallel and Sequential Agents created.")

✅ Parallel and Sequential Agents created.


In [17]:
runner = InMemoryRunner(agent=root_agent)
response = await runner.run_debug("Run the daily executive briefing on Tech, Health, and Finance")


 ### Created new session: debug_session_id

User > Run the daily executive briefing on Tech, Health, and Finance
FinanceResearcher > ## Daily Executive Briefing: November 12, 2025

**Technology:** The dominant tech trends revolve around **Artificial Intelligence**, particularly the rise of **agentic AI and context engineering**. This shift moves beyond simple chatbots to AI systems that can perform complex, multi-step tasks autonomously. The market implication is a significant demand for infrastructure orchestration, especially GPUs, to handle these advanced AI workloads. The future outlook suggests continued innovation in AI agents and specialized AI infrastructure.

**Health:** Key health trends include the broader integration of **AI for medical diagnosis and drug discovery**, along with the expansion of **remote healthcare and virtual hospitals**. The market is seeing increased investment in AI-powered diagnostic tools and wearables. The future outlook points to a more personalize

In [18]:
# This agent runs ONCE at the beginning to create the first draft.
initial_writer_agent = Agent(
    name="InitialWriterAgent",
    model="gemini-2.5-flash-lite",
    instruction="""Based on the user's prompt, write the first draft of a short story (around 100-150 words).
    Output only the story text, with no introduction or explanation.""",
    output_key="current_story", # Stores the first draft in the state.
)

print("✅ initial_writer_agent created.")

✅ initial_writer_agent created.


In [19]:
# This agent's only job is to provide feedback or the approval signal. It has no tools.
critic_agent = Agent(
    name="CriticAgent",
    model="gemini-2.5-flash-lite",
    instruction="""You are a constructive story critic. Review the story provided below.
    Story: {current_story}
    
    Evaluate the story's plot, characters, and pacing.
    - If the story is well-written and complete, you MUST respond with the exact phrase: "APPROVED"
    - Otherwise, provide 2-3 specific, actionable suggestions for improvement.""",
    output_key="critique", # Stores the feedback in the state.
)

print("✅ critic_agent created.")

✅ critic_agent created.


In [21]:
# This is the function that the RefinerAgent will call to exit the loop.
def exit_loop():
    """Call this function ONLY when the critique is 'APPROVED', indicating the story is finished and no more changes are needed."""
    return {"status": "approved", "message": "Story approved. Exiting refinement loop."}

print("✅ exit_loop function created.")

✅ exit_loop function created.


In [22]:
# This agent refines the story based on critique OR calls the exit_loop function.
refiner_agent = Agent(
    name="RefinerAgent",
    model="gemini-2.5-flash-lite",
    instruction="""You are a story refiner. You have a story draft and critique.
    
    Story Draft: {current_story}
    Critique: {critique}
    
    Your task is to analyze the critique.
    - IF the critique is EXACTLY "APPROVED", you MUST call the `exit_loop` function and nothing else.
    - OTHERWISE, rewrite the story draft to fully incorporate the feedback from the critique.""",
    
    output_key="current_story", # It overwrites the story with the new, refined version.
    tools=[FunctionTool(exit_loop)], # The tool is now correctly initialized with the function reference.
)

print("✅ refiner_agent created.")

✅ refiner_agent created.


In [23]:
# The LoopAgent contains the agents that will run repeatedly: Critic -> Refiner.
story_refinement_loop = LoopAgent(
    name="StoryRefinementLoop",
    sub_agents=[critic_agent, refiner_agent],
    max_iterations=2, # Prevents infinite loops
)

# The root agent is a SequentialAgent that defines the overall workflow: Initial Write -> Refinement Loop.
root_agent = SequentialAgent(
    name="StoryPipeline",
    sub_agents=[initial_writer_agent, story_refinement_loop],
)

print("✅ Loop and Sequential Agents created.")

✅ Loop and Sequential Agents created.


In [24]:
runner = InMemoryRunner(agent=root_agent)
response = await runner.run_debug("Write a short story about a lighthouse keeper who discovers a mysterious, glowing map")


 ### Created new session: debug_session_id

User > Write a short story about a lighthouse keeper who discovers a mysterious, glowing map
InitialWriterAgent > Elias had tended the lighthouse for thirty years, his life as predictable as the tide. Each night, the beam sliced through the darkness, a solitary eye watching over the restless sea. One blustery evening, while polishing the brass fixtures, his hand brushed against a loose stone in the wall. Behind it, nestled in the damp cavity, was a rolled-up parchment.

Unfurling it, Elias gasped. It wasn't paper, but a material that shimmered, etched with lines that pulsed with a soft, emerald light. Islands he'd never seen, constellations he didn't recognize, and a single, radiant 'X' at its center. The map seemed to hum, a silent song that vibrated in his bones. The familiar scent of salt and oil in the tower suddenly felt charged with an ancient magic. The sea outside, usually a comforting roar, now whispered secrets Elias was finally re